START OF THE FINAL PROJECT

All of the import statements

In [1]:
import pandas as pd
import transformers
from torch import torch_version
from torch import nn
import torch

1)LOAD THE DATASET

In [2]:
import pandas as pd
import string
import re 
# from config import CONFIG



class PREPROCESS:
    def __init__(self) -> None:
        df = pd.read_csv("spotify_millsongdata.csv")
        df = df[:9]
        df["prepText"] = df["text"].apply(self.cleanString)
        df = df[["artist", "song", "prepText"]]
        df["allText"] = df["artist"]+" "+df["song"] + " "+df["prepText"]
        # save the file 
        df.to_csv("preprocessed_songs.csv", index=False)
    
    def cleanString(self,sentence):
        sentence = sentence.translate(str.maketrans("","", string.punctuation)).strip().lower()
        sentence = re.sub(r"https?://\s+", "", sentence)
        sentence = re.sub(r"\b\d+\b",  "", sentence)
        sentence = re.sub(r" +"," ",sentence).replace("\n", " ").replace("\r", "").replace("\r\n", "")
        sentence = re.sub("\s+", " ", sentence)
        return sentence    




In [3]:
PREPROCESS()


In [4]:
from sentence_transformers import SentenceTransformer

import os
import numpy as np
from tqdm import tqdm
import faiss
class findEmbedding:
    def __init__(self) -> None:
        self.df = pd.read_csv("preprocessed_songs.csv")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        embeddings = self.computeEmbeddings()
        if not os.path.isdir("results"):
            os.mkdir("results")
        PATH = os.path.join("embedding.npy")
        with open( PATH, "wb") as fp:
            np.save(fp, embeddings)

    def computeEmbeddings(self):
        sentences = self.df["allText"].values
        embeddings = []
        for sentence in tqdm(sentences):
            embedding = self.model.encode(sentence)
            embeddings.append(embedding)
        return np.array(embeddings)

In [5]:
findEmbedding()

100%|██████████| 9/9 [00:00<00:00, 10.15it/s]


In [7]:
class recmmSystem:
    def __init__(self) -> None:
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.df = pd.read_csv("preprocessed_songs.csv")
        PATH = os.path.join( "embedding.npy")
        embeddings = np.load(PATH)
        dimension = embeddings.shape[1]
        quantize = faiss.IndexFlatL2(dimension)
        self.index = faiss.IndexIVFFlat(quantize, dimension, 50)

        if not self.index.is_trained:
            print(1)
            self.index.train(embeddings)
        
        if self.index.is_trained:
            print(0)
            self.index.add(embeddings)

        print("Total Number of embeddings index {}".format(self.index.ntotal))
    
    def nearestNeighbour(self, query, k):
        queryEmbeddings = self.model.encode([query])
        D, I = self.index.search(queryEmbeddings, k)
        artist_songs = dict()
        allRelatable = self.df.iloc[I[0]]
        for index, rows in allRelatable.iterrows():
            print(rows["artist"])
            artist_songs[rows["artist"]] = rows["song"]
        
        print("Query: ", query)
        print(artist_songs)
findEmbedding()
recmmSystem()

100%|██████████| 10/10 [00:00<00:00, 63.49it/s]


1


RuntimeError: Error in void faiss::Clustering::train_encoded(faiss::idx_t, const uint8_t *, const faiss::Index *, faiss::Index &, const float *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/Clustering.cpp:267: Error: 'nx >= k' failed: Number of training points (10) should be at least as large as number of clusters (50)

In [7]:
from torch import optim
from torch.utils.data import DataLoader, TensorDataset


class SongUserClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SongUserClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
num_users = 1  # Change this to the number of users you have
user_tensors = torch.eye(num_users)  # Create one-hot encoded tensors for users
labels = [1,1,1,1,1,0,0,0,0]
embeddings = np.load("embedding.npy")

embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
input_tensor = torch.cat((user_tensors_tensor.repeat(len(labels), 1), embeddings_tensor), dim=1)
input_size = input_tensor.shape[1]
hidden_size = 64
learning_rate = 0.001
batch_size = 4  # Adjust the batch size as needed
num_epochs = 100

# Create DataLoader for batch processing
dataset = TensorDataset(input_tensor, labels_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
model = SongUserClassifier(input_size, hidden_size,1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for inputs, labels in dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)

    # Print average loss for the epoch
    epoch_loss = total_loss / len(dataset)
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Extract learned user embeddings
user_embeddings = model.fc1.weight.data.numpy()

# Print learned user embeddings
# print("Learned User Embeddings:")
# print(user_embeddings)
def nearestNeighbour( queryEmbeddings, k):
     quantize = faiss.IndexFlatL2(queryEmbeddings.shape[1])
     index = faiss.IndexIVFFlat(quantize, queryEmbeddings.shape[1], 5)
     D, I = index.search(queryEmbeddings, k)
     print(-1)
     df = pd.read_csv("preprocessed_songs.csv")
     print(0)
     artist_songs = dict()
     allRelatable = df.iloc[I[0]]
     for index, rows in allRelatable.iterrows():
         print(1)
         print(rows["artist"])
         artist_songs[rows["artist"]] = rows["song"]
    
     print("Query: ", queryEmbeddings)
     print(artist_songs)
nearestNeighbour(user_embeddings,4)

-1
0
1
ABBA
1
ABBA
1
ABBA
1
ABBA
Query:  [[-0.02702218 -0.023104    0.0188639  ... -0.0647718  -0.05153453
  -0.033183  ]
 [-0.00563251 -0.02446619  0.0174718  ... -0.05098329 -0.08312376
  -0.02019404]
 [-0.01573327  0.01622819 -0.0430918  ...  0.03828778  0.02082782
   0.04206   ]
 ...
 [ 0.0363101  -0.01655955 -0.0487627  ... -0.00065791  0.04081779
   0.02449702]
 [ 0.0777607  -0.04043208 -0.03259625 ...  0.0696841   0.03718692
   0.00944527]
 [ 0.01299908 -0.00738772  0.01187073 ... -0.0252011  -0.00344736
   0.05499028]]
{'ABBA': 'Crazy World'}


/var/folders/bv/jp3prgcx78l1rxwtfc2pqmg40000gn/T/ipykernel_11845/2543545017.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  user_tensors_tensor = torch.tensor(user_tensors, dtype=torch.float32)
/Users/kamalgurbanov/PycharmProjects/cs389_recomendation_system/.venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [6]:

user_tensors = torch.eye(10) 
user_tensors

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])